In [1]:
from market_analysis.models import Candle, UserProfile, Symbol, SortedStocksList, StrategyTimestamp
# from market_analysis import views
from django.contrib.auth.models import User
from heystox_intraday.select_stocks_for_trading import *
import pandas as pd
# from heystox_trade.settings import *
from datetime import datetime, timedelta
from heystox_intraday.intraday_fetchdata import *
from market_analysis.tasks.stock_data_import_tasks import *
from market_analysis.tasks.day_trading_tasks import *
from django.core.cache import cache, caches
from datetime import time

In [2]:
# user = get_upstox_user("sonupal129@gmail.com")
# symbol = Symbol.objects.get(symbol="ashokley")
# # print(user.get_ohlc(user.get_instrument_by_symbol('NSE_EQ', 'bel'), OHLCInterval.Minute_1, datetime.strptime('12/12/2019', '%d/%m/%Y').date(), datetime.strptime('12/12/2019', '%d/%m/%Y').date()))
# user.get_master_contract("NSE_EQ")
# user.get_master_contract("NSE_INDEX")
# # cache_candles_data(user, symbol)
# # get_stock_current_candle("ashokley")

In [3]:
# def function_caller(start_time, end_time, func, *args, **kwargs):
#     """Call function on custom time with interval functionality using celery periodic task"""
#     current_time = datetime.now().time()
#     if start_time < current_time < end_time:
#         func(*args, **kwargs)

In [4]:
# def func(a,b):
#     a= 3
#     b = 4
#     print(a+b)
    
# start = time(9,15)
# end = time(15,30)
    
# function_caller(start,end, func, 3,4)

In [5]:
redis_cache = caches["redis"]
data = redis_cache.get("raju")
my_list = ["Sonu"]
if not data:
    print(1)

1


In [6]:
user = UserProfile.objects.first()
uuser = user.get_upstox_user()

In [23]:
symbol = Symbol.objects.get(symbol="jswsteel")
symbols = Symbol.objects.filter(modified_at__date=datetime.now().date())
from heystox_intraday.intraday_functions_strategy import *

In [60]:
# for symbol in symbols:
# #     print(symbol)
#     print(symbol.has_entry_for_long_short())

df = symbol.get_stock_live_data()
df

,candle_type,open_price,high_price,low_price,close_price,volume,total_buy_quantity,total_sell_quantity,date
0,M5,58.50,58.90,57.75,58.00,1189623,None,None,2019-12-18 09:15:00
1,M5,58.00,58.10,57.60,57.70,826799,None,None,2019-12-18 09:20:00
2,M5,57.80,58.50,57.70,58.50,676335,None,None,2019-12-18 09:25:00
3,M5,58.45,58.45,58.00,58.15,695544,None,None,2019-12-18 09:30:00
4,M5,58.20,58.20,57.80,57.90,271663,None,None,2019-12-18 09:35:00
...,...,...,...,...,...,...,...,...,...
145,M5,58.95,59.15,58.90,59.10,454322,None,None,2019-12-19 15:05:00
146,M5,59.10,59.25,59.05,59.15,269974,None,None,2019-12-19 15:10:00
147,M5,59.15,59.20,59.00,59.10,574187,None,None,2019-12-19 15:15:00
148,M5,59.10,59.25,59.05,59.20,743110,None,None,2019-12-19 15:20:00
